# Reading the data 

---

Lately, the _.fits_ data was transformed into _.csv_ and saved in database/csv_files. We are going to import a light curve sample and work with it.

In [2]:
# FILE_PATH = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/exoplanets_confirmed/resampled_files/RESAMPLED_EN2_STAR_CHR_0101086161_20070516T060226_20071005T074409.csv'
FILE_PATH = 'C:/Users/guisa/Google Drive/01 - Iniciação Científica/02 - Datasets/exoplanets_confirmed/resampled_files/RESAMPLED_EN2_STAR_CHR_0105833549_20080415T231048_20080907T224903.csv'

In [3]:
import pandas as pd

data_sample = pd.read_csv(FILE_PATH)
display(data_sample)

DATE      WHITEFLUX
0      2008-04-16 11:17:01.652367  425517.134412
1      2008-04-16 11:30:51.915174  426870.998701
2      2008-04-16 11:44:42.177981  426445.431775
3      2008-04-16 11:58:32.440788  426497.707393
4      2008-04-16 12:12:22.703595  426530.518909
...                           ...            ...
15045  2008-09-08 01:05:25.581928  426348.291249
15046  2008-09-08 01:19:15.844734  424517.308637
15047  2008-09-08 01:33:06.107541  423893.013298
15048  2008-09-08 01:46:56.370348  423597.101577
15049  2008-09-08 02:00:46.633155  423233.988598

[15050 rows x 2 columns]

In [4]:
import numpy as np

x = data_sample.DATE.to_numpy()
y = data_sample.WHITEFLUX.to_numpy()

# Bessel Transfer Function

---

The transfer function of a Bessel lowpass filter is defined by

$$ H(s) = \frac{\theta_n (0) }{\theta_n (s/ \omega_0)}  \\ $$ 

$$ \theta_n (s) = \sum_{k=0}^{n} a_k s^k $$

$$ a_k = \frac{(2n-k)!}{2^{n-k}k!(n-k)!} \\ $$




# Filtering

### Parameters

In [5]:
order = 2
cutoff_freq = 0.9

### Control lib...

In [6]:
from control import *

In [7]:
### Computing ak

from math import factorial

coef = []
i = 0
while i <= order:
  ak = (factorial(2*order - i)) / ( 2**(order - i)*factorial(i)*factorial(order - i) )
  # print(ak)
  coef.append(ak)
  i += 1

print(coef)

[3.0, 3.0, 1.0]


In [8]:
### Computing θn(s)

s = TransferFunction.s
theta_array = []
k = 0
for k in range(order+1):
  theta_n = coef[k] * (s**k)
  theta_array.append(theta_n)
  # numerical_numerator = coef[0]
  # print(theta_n)

print(theta_array[0])
print(theta_array[1])
print(theta_array[2])
#print(theta_array[3])


3
-
1


3 s
---
 1


s^2
---
 1



In [9]:
### Computing H(s)

coef_numerator = theta_array[0]

list_denominator = theta_array[:]

In [10]:
denominator = 0
for item in list_denominator:
  denominator += item

denominator

TransferFunction(array([1., 3., 3.]), array([1.]))

In [11]:
### Preenchendo transfer function

G = coef_numerator / denominator

print(G)
print(type(G))


      3
-------------
s^2 + 3 s + 3

<class 'control.xferfcn.TransferFunction'>


### Applying 



In [12]:
import numpy as np

In [13]:
def bessel(array, fourier_transform, cutoff_freq, order):

  # Extracting features from signal
  n_time = len(array)
  D0 = cutoff_freq * n_time
  xc = n_time

  # Creating the bessel transfer function array
  len_filter = len(fourier_transform)
  filter = np.zeros(len_filter)
  i=0

  for i in range(len_filter):
    filter[i] = np.real(evalfr(G, ( np.abs(i-(xc-1.0))/D0 )))

  return filter

# Choosing the best Bessel parameters

---

On `filters` module, we can pass `bessel` on the `filter_technique` and it will makes all the procedures for the Bessel Lowpass filter.

In [14]:
from tools import *

Loading BokehJS ...

## Plotting different cutoff frequencies and orders

In [15]:
orders = [1, 2, 3, 4, 5, 6]
cutoff_freqs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [16]:
import time
t_o = time.time()

count = 0
for cutoff_freq in cutoff_freqs:
  for order in orders:  
        # Filtering the curve
        Filter = filters.FrequencyDomainFiltering()
        Filter.filter(array=y, filter_technique='bessel', numExpansion=70, cutoff_freq=cutoff_freq, order=order)
        y_filtered = Filter.getFiltered
        y_filtered += (y.mean() - y_filtered.mean())
       
        # Salving filtered data
        PATH_DIR = 'C:/Users/guisa/Google Drive/01 - Iniciação Científica/02 - Datasets/exoplanets_confirmed/filters/bessel/order'+str(order)+'/cutoffFreq0'+str(int(cutoff_freq*10))       
        filters.export_results_csv(PATH_DIR, 'bessel', cutoff_freq, order)
        
        count +=1 

        
t_f = time.time()
print("It takes:", round(t_f-t_o, 2), "seconds to save all")

Saving filtered data for bessel, order = 1 and cutoff frequency = 0.1
All files have been save sucessefuly

Saving filtered data for bessel, order = 2 and cutoff frequency = 0.1
All files have been save sucessefuly

Saving filtered data for bessel, order = 3 and cutoff frequency = 0.1
All files have been save sucessefuly

Saving filtered data for bessel, order = 4 and cutoff frequency = 0.1
All files have been save sucessefuly

Saving filtered data for bessel, order = 5 and cutoff frequency = 0.1
All files have been save sucessefuly

Saving filtered data for bessel, order = 6 and cutoff frequency = 0.1
All files have been save sucessefuly

Saving filtered data for bessel, order = 1 and cutoff frequency = 0.2
All files have been save sucessefuly

Saving filtered data for bessel, order = 2 and cutoff frequency = 0.2
All files have been save sucessefuly

Saving filtered data for bessel, order = 3 and cutoff frequency = 0.2
All files have been save sucessefuly

Saving filtered data for bes

## Plotting some filters

In [17]:
order = 2
cutoff_freq = 0.1

Filter = filters.FrequencyDomainFiltering()
Filter.filter(array=y, filter_technique='bessel', numExpansion=70, cutoff_freq=cutoff_freq, order=order)
y_filtered = Filter.getFiltered
y_filtered += (y.mean() - y_filtered.mean())
title = f"Bessel filter with Order = {order} and Cutoff frequency = {cutoff_freq}"
viz.view_filter_results(x, y, x, y_filtered, title=title)

In [18]:
order = 4
cutoff_freq = 0.3

Filter = filters.FrequencyDomainFiltering()
Filter.filter(array=y, filter_technique='bessel', numExpansion=70, cutoff_freq=cutoff_freq, order=order)
y_filtered = Filter.getFiltered
y_filtered += (y.mean() - y_filtered.mean())
title = f"Bessel filter with Order = {order} and Cutoff frequency = {cutoff_freq}"
viz.view_filter_results(x, y, x, y_filtered, title=title)

In [19]:
order = 5
cutoff_freq = 0.2

Filter = filters.FrequencyDomainFiltering()
Filter.filter(array=y, filter_technique='bessel', numExpansion=70, cutoff_freq=cutoff_freq, order=order)
y_filtered = Filter.getFiltered
y_filtered += (y.mean() - y_filtered.mean())
title = f"Bessel filter with Order = {order} and Cutoff frequency = {cutoff_freq}"
viz.view_filter_results(x, y, x, y_filtered, title=title)